# Calculate original Ikeda for one ship

In [ ]:
#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator



In [ ]:
db = database.get_db()

df_simplified_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', limit_score=0.5, 
                             exclude_table_name='rolldecay_exclude')

df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.9, 
                             exclude_table_name='rolldecay_exclude')

df_rolldecay_cubic = database.load(rolldecay_table_name='rolldecay_cubic_b', limit_score=0.9, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
df_rolldecay.head()

### Selecting one ship

In [ ]:
project_number=20157375

In [ ]:
mask = (df_rolldecay['project_number'] == project_number)
df_rolldecay.loc[mask]

### Ballast loading condition

In [ ]:
TF=9.4
#TF=11.7
mask = ((df_rolldecay['project_number'] == project_number) &
        (df_rolldecay['TF'] == TF)
       )
df_ballast = df_rolldecay.loc[mask].copy()
df_ballast

In [ ]:
mask = ((df_rolldecay_cubic['project_number'] == project_number) &
        (df_rolldecay_cubic['TF'] == TF)
       )
df_ballast_cubic = df_rolldecay_cubic.loc[mask].copy()
df_ballast_cubic

In [ ]:
df_ballast_simplified_ikeda = df_simplified_ikeda.loc[df_ballast.index].copy()
df_ballast_simplified_ikeda

In [ ]:
meta_data=df_ballast.iloc[0]
meta_data_cubic=df_ballast_cubic.loc[meta_data.name]
meta_data_ikeda=df_ballast_simplified_ikeda.loc[meta_data.name]

db_run = db.session.query(Run).get(int(meta_data.name))
df = database.load_run(db_run)

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25), phi1d_start_tolerance=0.015)
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
#    ('scaler',scaler),  # Is froude scaling a good idea??
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer)
]
        
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X = preprocessor.transform(df)

In [ ]:
direct_estimator = EstimatorQuadraticB.load(data=meta_data, X=X)
direct_estimator_cubic = EstimatorCubic.load(data=meta_data_cubic, X=X)
ikeda_estimator = IkedaQuadraticEstimator.load(data=meta_data_ikeda, X=X)

fig,ax=plt.subplots()
direct_estimator.plot_fit(ax=ax)
fig,ax=plt.subplots()
direct_estimator_cubic.plot_fit(ax=ax)
fig,ax=plt.subplots()
ikeda_estimator.plot_fit(ax=ax)

In [ ]:
meta_data_ikeda['B_e']

In [ ]:
meta_data['B_1']

In [ ]:
meta_data['B_2']

## Run Scores2

In [ ]:
from pyscores2.indata import Indata
from pyscores2.runScores2 import Calculation
from pyscores2.output import OutputFile

In [ ]:
indata_file_name = 'data207_name%s_loading_condition_id%s' % (meta_data.name, meta_data.loading_condition_id)
indata_path = os.path.join('scores2', indata_file_name)
indata = Indata()
indata.open(indata_path)

indata.kxx=meta_data['KXX']
indata.displacement=meta_data['Volume']
indata.rho=1000
indata.zcg=meta_data['kg']-meta_data['TA'] #(important)
indata.lcb=meta_data['lcg']

indata.waveDirectionMin=0
indata.waveDirectionMax=180
indata.waveDirectionIncrement=90
indata.speedMin=0
indata.speedMax=18.6
indata.speedIncrement=18.6


In [ ]:
-(meta_data['lcg']-meta_data['lpp']/2)

In [ ]:
outdata_directory=r'scores2/result'
if not os.path.exists(outdata_directory):
    os.mkdir(outdata_directory)

calculation = Calculation(outDataDirectory=outdata_directory)
calculation.run(indata=indata)

In [ ]:
indata.displacement

In [ ]:
output_file = OutputFile(filePath=calculation.outDataPath)
df = output_file.get_result()

fig,ax=plt.subplots()
for index, group in df.groupby(by=['speed','wave direction'], sort=False):
    group.plot(x='frequencies', y='rollAmplitude', style='o-', label=index, ax=ax)
    
ax.grid(True)
ax.legend();
ax.set_ylabel('Roll');

In [ ]:
df_roll_damping = output_file.get_roll_damping()
df_roll_damping

In [ ]:
df_roll_damping.set_index(['speed','wave_direction'], inplace=True)

In [ ]:
scores_damping = df_roll_damping.loc[0,0]
scores_damping

In [ ]:
BW_scores = scores_damping['calculated_wave_damping_in_roll']

In [ ]:
meta_data['omega0']/np.sqrt(meta_data['scale_factor'])

In [ ]:
import rolldecayestimators.simplified_ikeda as si

In [ ]:
scale_factor = meta_data['scale_factor']
LPP=meta_data['lpp']
Beam=meta_data['beam']
DRAFT=(meta_data['TA']+meta_data['TF'])/2
Volume=meta_data['Volume']
CB=Volume/(Beam*DRAFT*LPP)
CMID=meta_data['A0']
OG=-meta_data['kg'] + DRAFT
PHI=5
lBK=meta_data['BKL']
bBK=meta_data['BKB']
OMEGA=meta_data['omega0']
V=meta_data['ship_speed']*1.852/3.6

#Froude scale:
LPP/=scale_factor
Beam/=scale_factor
DRAFT/=scale_factor
Volume/=scale_factor**3
OG/=scale_factor
lBK/=scale_factor
bBK/=scale_factor
#OMEGA=meta_data['omega0']
V/=np.sqrt(scale_factor)


B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT, BLHAT = si.calculate_roll_damping(LPP=LPP,Beam=Beam,CB=CB,
                                        CMID=CMID,OG=OG,PHI=PHI,lBK=lBK,bBK=bBK,OMEGA=OMEGA,
                                        DRAFT=DRAFT, V=V, verify_input=False, limit_inputs=True)

In [ ]:
si_result = pd.Series(name='SI-method')
si_result['B44HAT']=B44HAT
si_result['BFHAT']=BFHAT
si_result['BWHAT']=BWHAT
si_result['BEHAT']=BEHAT
si_result['BBKHAT']=BBKHAT
si_result['BLHAT']=BLHAT

In [ ]:
si_result.plot(kind='bar')

In [ ]:
BWHAT

In [ ]:
from rolldecayestimators import equations
from rolldecayestimators import symbols, lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import sympy as sp

In [ ]:
lambdas.B44_hat_equation

In [ ]:
g=9.81
rho=1000
BW_hat_scores=lambdas.B_hat_lambda(B=BW_scores, Disp=Volume*scale_factor**3, beam=Beam*scale_factor, 
                                   g=g, rho=rho)
BW_hat_scores

In [ ]:
BWHAT/BW_hat_scores

In [ ]:
BFHAT+BW_hat_scores+BBKHAT+BLHAT

In [ ]:
B_e = lambdas.B_e_lambda(B_1=meta_data['B_1'], B_2=meta_data['B_2'], phi_a=np.deg2rad(8), 
                   omega0=meta_data['omega0'])


B_e_hat = lambdas.B_e_hat_lambda(B_e=B_e, Disp=Volume, beam=Beam, 
                                 g=g, rho=rho)
B_e_hat

In [ ]:
B44HAT

## Ikeda

In [ ]:
from rolldecayestimators import ikeda_speed

In [ ]:
inputs = pd.DataFrame()
N=20
speeds=np.linspace(df_ballast['ship_speed'].min(),df_ballast['ship_speed'].max(),N)*1.852/3.6/np.sqrt(scale_factor)
inputs['V'] = speeds  # Ship speed [m/s]
inputs['w'] = w = meta_data['omega0']  # roll frequency [rad/s]
inputs['d'] = DRAFT  # Draught of hull [m]
BW_scores_model=BW_scores/(scale_factor**4/np.sqrt(scale_factor))
inputs['Bw0'] = BW_scores_model  # Zero speed wave damping [Nm*s/rad]
inputs['fi_a'] = np.deg2rad(8) # Roll amplitude [rad]
inputs['B'] = Beam # Breadth of hull [m]
inputs['A'] = CMID*Beam*DRAFT # Area of cross section of hull [m2]
inputs['bBK'] = bBK  #breadth of Bilge keel [m] !!(=height???)
inputs['R'] = 0.1 # Bilge Radis [m]
inputs['OG'] = OG # distance from roll axis to still water level [m]
inputs['Ho'] = Beam/(2*DRAFT)  # half breadth to draft ratio B/(2*d) [-]
inputs['ra'] = ra = rho  # density of water (1025) [kg/m3]
inputs['Cb'] =CB  # Block coeff [-]
inputs['L']=LPP  # Ship length [m]
inputs['LBK']=lBK  # Bilge keel length [m]
inputs['Bw_div_Bw0_max'] = np.inf  # maxmum allowed difference between Bw0 and at speed.
inputs['visc']=1.15*10**-6
inputs['g']=g=9.81
results = inputs.apply(ikeda_speed.calculate_B44_series,axis=1)

In [ ]:
results_hat = results.apply(func=lambdas.B_hat_lambda,Disp=Volume, beam=Beam, 
                                   g=g, rho=rho)

fig,ax=plt.subplots()
results_hat.plot.area(y = ['B_BK','B_L','B_W','B_F'], ax=ax)


In [ ]:
results_hat

In [ ]:
renamings = {
    'B44HAT':'B_44',
    'BFHAT':'B_F',
    'BWHAT':'B_W',
    'BEHAT':'B_E',
    'BBKHAT':'B_BK',
    'BLHAT':'B_L',
}

si_result.rename(index=renamings, inplace=True)

In [ ]:
df_results = pd.DataFrame()
df_results=df_results.append(si_result)
i_result=results_hat.iloc[0]
i_result.name='Ikeda-method'
df_results=df_results.append(i_result)

df_results.transpose().plot(kind='bar')